In [1]:
import sys
from PyQt5.QtWidgets import QMainWindow, QApplication, QWidget, QAction, QTableWidget,QTableWidgetItem,QVBoxLayout
from PyQt5 import uic, QtCore
from PyQt5.QtGui import QIcon, QPixmap
import psycopg2

In [2]:
qtCreatorFile = "appUI.ui" # Enter file here.
Ui_MainWindow, QtBaseClass = uic.loadUiType(qtCreatorFile)

In [3]:
class milestone1(QMainWindow):
    def __init__(self):
        super(milestone1, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)
        self.loadStateList()
        self.loadCatList()
        self.ui.stateList.currentTextChanged.connect(self.stateChanged)
        self.ui.cityList.itemSelectionChanged.connect(self.cityChanged)
        self.ui.zipList.itemSelectionChanged.connect(self.zipChanged)
        self.ui.bname.textChanged.connect(self.getBusinessNames)
        self.ui.businesses.itemSelectionChanged.connect(self.displayBusinessCity)
        self.ui.pushButton.clicked.connect(self.buttpress)
        self.ui.b2.clicked.connect(self.b2press)
        
    def execQuery(self,sql_str):
        try:
            conn = psycopg2.connect("dbname='milestone2DB' user=postgres host='localhost' password='sudo'")
        except:
            print('unable to connect to database')
        cur = conn.cursor()
        cur.execute(sql_str)
        result = cur.fetchall()
        conn.commit()
        conn.close()
        return result
        
    def loadStateList(self):
        self.ui.stateList.clear()
        sql_str = "SELECT DISTINCT state FROM businesstable ORDER BY state;"
        try:
            results = self.execQuery(sql_str)
            for row in results:
                self.ui.stateList.addItem(row[0])
        except:
            print("Query failed")
        self.ui.stateList.setCurrentIndex(-1)
        self.ui.stateList.clearEditText()
        
    def stateChanged(self):
        self.ui.cityList.clear()
        state = self.ui.stateList.currentText()
        if (self.ui.stateList.currentIndex() >= 0):
            sql_str = "SELECT DISTINCT city FROM businesstable WHERE state='" + state +"' ORDER BY city;"
            try:
                results = self.execQuery(sql_str)
                for row in results:
                    self.ui.cityList.addItem(row[0])
            except:
                print("Query failed")
                
            
            for i in reversed(range(self.ui.businessTable.rowCount())):
                self.ui.businessTable.removeRow(i)
                
            sql_str = "SELECT name, city, state FROM businesstable WHERE state='" + state +"' ORDER BY name;"
            try:
                results = self.execQuery(sql_str)
                style = "::section {""background-color: #f3f3f3;}"
                self.ui.businessTable.horizontalHeader().setStyleSheet(style)
                self.ui.businessTable.setColumnCount(len(results[0]))
                self.ui.businessTable.setRowCount(len(results))
                self.ui.businessTable.setHorizontalHeaderLabels(['Business Name', 'City', 'State'])
                self.ui.businessTable.resizeColumnsToContents()
                self.ui.businessTable.setColumnWidth(0, 300)
                self.ui.businessTable.setColumnWidth(1, 100)
                self.ui.businessTable.setColumnWidth(2, 50)
                currentRowCount = 0
                for row in results:
                    for colCount in range (0, len(results[0])):
                        self.ui.businessTable.setItem(currentRowCount, colCount, QTableWidgetItem(row[colCount]))
                    currentRowCount += 1
            except:
                print("Query failed")
           
                
    def cityChanged(self):
        if (self.ui.stateList.currentIndex() >= 0) and (len(self.ui.cityList.selectedItems()) > 0):
            state = self.ui.stateList.currentText()
            city = self.ui.cityList.selectedItems()[0].text()
            sql_str = "SELECT name, city, state FROM businesstable WHERE state='" + state +"' AND city='" + city +"' ORDER BY name;"
            try:
                results = self.execQuery(sql_str)
                style = "::section {""background-color: #f3f3f3;}"
                self.ui.businessTable.horizontalHeader().setStyleSheet(style)
                self.ui.businessTable.setColumnCount(len(results[0]))
                self.ui.businessTable.setRowCount(len(results))
                self.ui.businessTable.setHorizontalHeaderLabels(['Business Name', 'City', 'State'])
                self.ui.businessTable.resizeColumnsToContents()
                self.ui.businessTable.setColumnWidth(0, 300)
                self.ui.businessTable.setColumnWidth(1, 100)
                self.ui.businessTable.setColumnWidth(2, 50)
                currentRowCount = 0
                for row in results:
                    for colCount in range (0, len(results[0])):
                        self.ui.businessTable.setItem(currentRowCount, colCount, QTableWidgetItem(row[colCount]))
                    currentRowCount += 1
            except:
                print("Query failed")
                
            self.ui.zipList.clear()
            
            sql_str = "SELECT DISTINCT zipcode FROM businesstable WHERE state='" + state +"' AND city='" + city + "';"
            try:
                results = self.execQuery(sql_str)
                for row in results:
                    self.ui.zipList.addItem(row[0])
            except:
                print("Query failed")
                
    def getBusinessNames(self):
        self.ui.businesses.clear()
        businessname = self.ui.bname.toPlainText()
        sql_str = "SELECT name FROM businesstable WHERE name LIKE '%" + businessname + "%' ORDER BY name;"
        try:
            results = self.execQuery(sql_str)
            for row in results:
                self.ui.businesses.addItem(row[0])
        except:
            print("Query failed")
        
    def displayBusinessCity(self):
        businessname = self.ui.businesses.selectedItems()[0].text()
        sql_str = "SELECT city FROM businesstable WHERE name='" + businessname.replace("'", "''") + "';"
        #print(sql_str)
        try:
            results = self.execQuery(sql_str)
            print(results)
            self.ui.bcity.setText(results[0][0])
        except:
            print("Query failed")
            
    def zipChanged(self):
        if (self.ui.stateList.currentIndex() >= 0) and (len(self.ui.cityList.selectedItems()) > 0) and (len(self.ui.zipList.selectedItems()) > 0):
            state = self.ui.stateList.currentText()
            city = self.ui.cityList.selectedItems()[0].text()
            zipcode = self.ui.zipList.selectedItems()[0].text()
            sql_str = "SELECT name, city, state FROM businesstable WHERE state='" + state +"' AND city='" + city +"' AND zipcode ='" + zipcode +"'ORDER BY name;"
            try:
                results = self.execQuery(sql_str)
                style = "::section {""background-color: #f3f3f3;}"
                self.ui.businessTable.horizontalHeader().setStyleSheet(style)
                self.ui.businessTable.setColumnCount(len(results[0]))
                self.ui.businessTable.setRowCount(len(results))
                self.ui.businessTable.setHorizontalHeaderLabels(['Business Name', 'City', 'State'])
                self.ui.businessTable.resizeColumnsToContents()
                self.ui.businessTable.setColumnWidth(0, 300)
                self.ui.businessTable.setColumnWidth(1, 100)
                self.ui.businessTable.setColumnWidth(2, 50)
                currentRowCount = 0
                for row in results:
                    for colCount in range (0, len(results[0])):
                        self.ui.businessTable.setItem(currentRowCount, colCount, QTableWidgetItem(row[colCount]))
                    currentRowCount += 1
            except:
                print("Query failed")
                
            #self.ui.zipList.clear()
            
            #sql_str = "SELECT DISTINCT zipcode FROM businesstable WHERE state='" + state +"' AND city='" + city + "';"
            #try:
             #   results = self.execQuery(sql_str)
              #  for row in results:
              #      self.ui.zipList.addItem(row[0])
            #except:
              #  print("Query failed")
                
                
                
            sql_str = "SELECT count(business_id) FROM businesstable WHERE zipcode ='" + zipcode + "';"
            #print(sql_str)
            try:
                results = self.execQuery(sql_str)
                #print(results)
                self.ui.numbusiness.setText(str(results[0][0]))
            except:
                print("Query failed")
            
            sql_str = "SELECT population FROM population WHERE zip = '" + zipcode + "';"
            #print(sql_str)
            try:
                results = self.execQuery(sql_str)
                #print(results)
                self.ui.pname.setText(str(results[0][0]))
            except:
                print("Query failed")
                
            sql_str = "SELECT count(category_name) as cname, category_name FROM \
                        (SELECT category_name, zipcode, state, city FROM businesstable, categories \
                        WHERE businesstable.business_id = categories.business_id) as temp \
                        WHERE state='" + state +"' AND city='" + city +"' AND zipcode = '" + zipcode +"' \
                        GROUP BY category_name ORDER BY cname DESC"
            
            try:
                results = self.execQuery(sql_str)
                style = "::section {""background-color: #f3f3f3;}"
                self.ui.catTable.horizontalHeader().setStyleSheet(style)
                self.ui.catTable.setColumnCount(len(results[0]))
                self.ui.catTable.setRowCount(len(results))
                self.ui.catTable.setHorizontalHeaderLabels(['# of businesses', 'category'])
                self.ui.catTable.resizeColumnsToContents()
                self.ui.catTable.setColumnWidth(0, 100)
                self.ui.catTable.setColumnWidth(1, 100)
                currentRowCount = 0
                for row in results:
                    for colCount in range (0, len(results[0])):
                        self.ui.catTable.setItem(currentRowCount, colCount, QTableWidgetItem(str(row[colCount])))
                    currentRowCount += 1
            except:
                print("Query failed")
                
            
            self.ui.catList.clear()
            sql_str = "SELECT DISTINCT category_name FROM categories, businesstable \
            WHERE businesstable.business_id=categories.business_id \
            AND state='" + state +"' AND city='" + city +"' AND zipcode ='" + zipcode +"' ORDER BY category_name;"
            try:
                results = self.execQuery(sql_str)
                for row in results:
                    self.ui.catList.addItem(row[0])
            except:
                print("Query failed")
            
                

    def loadCatList(self):        
        sql_str = "SELECT DISTINCT category_name FROM categories ORDER BY category_name;"
        try:
            results = self.execQuery(sql_str)
            for row in results:
                self.ui.catList.addItem(row[0])
        except:
            print("Query failed")
            
    def buttpress(self):
        if (self.ui.stateList.currentIndex() >= 0) and (len(self.ui.cityList.selectedItems()) > 0) and (len(self.ui.zipList.selectedItems()) > 0):
            state = self.ui.stateList.currentText()
            city = self.ui.cityList.selectedItems()[0].text()
            zipcode = self.ui.zipList.selectedItems()[0].text()
            category = self.ui.catList.selectedItems()[0].text()
            sql_str = "SELECT name, city, state FROM businesstable, categories WHERE businesstable.business_id=categories.business_id\
            AND state='" + state +"' AND city='" + city +"' AND zipcode ='" + zipcode +"' AND category_name ='" + category +"'ORDER BY name;"
            try:
                #print(sql_str)
                results = self.execQuery(sql_str)
                style = "::section {""background-color: #f3f3f3;}"
                self.ui.businessTable.horizontalHeader().setStyleSheet(style)
                self.ui.businessTable.setColumnCount(len(results[0]))
                self.ui.businessTable.setRowCount(len(results))
                self.ui.businessTable.setHorizontalHeaderLabels(['Business Name', 'City', 'State'])
                self.ui.businessTable.resizeColumnsToContents()
                self.ui.businessTable.setColumnWidth(0, 300)
                self.ui.businessTable.setColumnWidth(1, 100)
                self.ui.businessTable.setColumnWidth(2, 50)
                currentRowCount = 0
                for row in results:
                    for colCount in range (0, len(results[0])):
                        self.ui.businessTable.setItem(currentRowCount, colCount, QTableWidgetItem(row[colCount]))
                    currentRowCount += 1
            except:
                print("Query failed")
                
    def b2press(self):
        if (self.ui.stateList.currentIndex() >= 0) and (len(self.ui.cityList.selectedItems()) > 0) and (len(self.ui.zipList.selectedItems()) > 0):
            state = self.ui.stateList.currentText()
            city = self.ui.cityList.selectedItems()[0].text()
            zipcode = self.ui.zipList.selectedItems()[0].text()
            category = self.ui.catList.selectedItems()[0].text()
            sql_str = "SELECT name, city, state, businesstable.reviewrating FROM Businesstable, \
(SELECT businesstable.business_id, category_name, reviewrating \
FROM businesstable, categories \
WHERE businesstable.business_id=categories.business_id \
AND state='" + state +"' AND city='" + city +"' AND zipcode ='" + zipcode +"' AND category_name ='" + category +"') as temp, \
(SELECT AVG(reviewrating) as avgR \
FROM businesstable, categories \
WHERE businesstable.business_id=categories.business_id \
AND state='" + state +"' AND city='" + city +"' AND zipcode ='" + zipcode +"' AND category_name ='" + category +"') as review \
WHERE businesstable.business_id=temp.business_id AND businesstable.reviewrating > review.avgR \
ORDER BY businesstable.reviewrating DESC;"
            try:
                results = self.execQuery(sql_str)
                style = "::section {""background-color: #f3f3f3;}"
                self.ui.poptable.horizontalHeader().setStyleSheet(style)
                self.ui.poptable.setColumnCount(len(results[0]))
                self.ui.poptable.setRowCount(len(results))
                self.ui.poptable.setHorizontalHeaderLabels(['Business Name', 'City', 'State', 'review Rating'])
                self.ui.poptable.resizeColumnsToContents()
                self.ui.poptable.setColumnWidth(0, 300)
                self.ui.poptable.setColumnWidth(1, 100)
                self.ui.poptable.setColumnWidth(2, 50)
                self.ui.poptable.setColumnWidth(3, 100)
                currentRowCount = 0
                for row in results:
                    for colCount in range (0, len(results[0])):
                        self.ui.poptable.setItem(currentRowCount, colCount, QTableWidgetItem(str(row[colCount])))
                    currentRowCount += 1
            except:
                print("Query failed")
        

In [4]:
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = milestone1()
    window.show()
    sys.exit(app.exec_())

[('Scottsdale',)]


SystemExit: 0

C:\Users\khali\anaconda3\envs\data451\lib\site-packages\IPython\core\interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
